## Langchain using ChatGPT models - RAG

### Import packages

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter  
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.vectorstores import Weaviate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.embeddings import OpenAIEmbeddings

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline, BitsAndBytesConfig

from typing import List, Optional, Union
import weaviate
from weaviate.embedded import EmbeddedOptions
import weaviate
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv,find_dotenv

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
def data_loader(file_path: str, chunk_size: int = 500, chunk_overlap: int = 50) -> Union[List[str], None]:
    """
    Load data from a file, split it into chunks, and return the chunks.

    Parameters:
    - file_path (str): The path to the file containing the data.
    - chunk_size (int): The size of each data chunk. Default is 500.
    - database (int): The overlap between consecutive chunks. Default is 50.

    Returns:
    - list: A list of data chunks.
    """
    try:
        loader = TextLoader(file_path)
        documents = loader.load()

        # Chunk the data
        text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        chunks = text_splitter.split_documents(documents)
        
        print("Data loaded to vector database successfully")
        return chunks
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None 
    
    
def create_retriever(chunks):
    try:
        # Load OpenAI API key from .env file
        load_dotenv(find_dotenv())
        #  Setup vector database
        client = weaviate.Client(embedded_options=EmbeddedOptions())

        # Populate vector database using embeddings from the Hugging Face model
        vectorstore = Weaviate.from_documents(
            client=client,
            documents=chunks,
            embedding=OpenAIEmbeddings(),  # Use the model's encode function for embeddings
            by_text=False
        )

        # Define vectorstore as retriever to enable semantic search
        retriever = vectorstore.as_retriever()
        print("Retriever created successfully.")

        return retriever

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

### Load the data and create chunks

In [3]:
chuncks = data_loader("../prompts/context.txt")
len(chuncks)

Data loaded to vector database successfully


9

### Create the retriver

In [4]:
retriever = create_retriever(chuncks)

Started /home/babi/.cache/weaviate-embedded: process ID 19560


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-02-01T14:50:42+03:00"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-02-01T14:50:42+03:00"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-02-01T14:50:42+03:00"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-02-01T14:50:42+03:00"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50060","time":"2024-02-01T14:50:42+03:00"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://12

Retriever created successfully.


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


In [5]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [6]:
retriever.get_relevant_documents("ኢትዮጵያ")

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[Document(page_content='ኢትዮጵያዊነት ኢትዮጵያዊነት ብዙ የተለያዩ ህብረተሰቦች የተዋህዱበት አካል ክፍል\nመሆን ነው። ኢትዮጵያዊነት ከጎሰኝኛት በላይና ውጭ የሆነ አጠቃላይ ማንነት ነው።\nኢትዮጵያዊነት የዚች ውጥንቅጥ መሬት ባለቤትነት ነው፣ ኢትዮጵያዊነት የደጋው\nብርድና የቆላው ሙቀት የሚገናኙበት፣ የደጋው ዝናብም ከቆላው ወንዝ ጋር\n\nየተዛመደበት ሃይል ነው። ኢትዮጵያዊነት ለሙሴ፣ ለክርስቶስና ለመሀመድ የሚጨሰው \nእጣን በእርገት መጥቶ የሚደባለቅበት እምነት ነው። ኢትዮጵያዊነት በቄጤማ ጉዝጓዝ\nላይ የሚታየው መተሳሰብና መፈቃቀር ነው። ኢትዮጵያዊነት የአስተዋይነት፣የሚዛናዊነት \nየጨዋነት ባሕርይ ነው። ኢትዮጵያዊነት ረዢምና ተፅፎ ያላለቀ ታሪክ ነው፤ ብዙ ሰዎችም\nየተሰዉለት ስሜት ነው።', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ኢትዮጵያዊነት ስቃይና መከራ ነው፤ ኢትዮጵያዊነት ኩራትና\nክብርም ነው። ኢትዮጵያዊነት ጭቆናና ጥቃት ነው፤ ኢትዮጵያዊነት አሻፈረኝና እምቢ \nባይነትም ነው። የኢትዮጵያዊነት ኣላማ ያለችውንና የኖረችውን ኢትዮጵያን እንደገና \nለመፍጠር አይደለም። የኢትዮጵያ ኣላማ ለኢትዮጵያዊያን በሙሉ የሚስማማ እድገታቸውንም\nየሚያፋጥንና፣ህብረታቸውን የሚያጠነክር ሥርኣትን መፍጠር ነው። የኢትዮጵያዊነት ችግር', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ኢትዮጵያ አይደለችም፣ችግሩ የሥርኣት ችግር ነው። የቤቴ መቃጠል ለትኋኑ በጀኝ እንዳለው\nቂል እንዳንሆን። ኢትዮጵያዊነትን መካድ የራስን ማንነት መካድ ብቻ አይደለም። አባቶችንና\nእናቶችን፣ቅድም አያቶችን ከነቅርሳቸው መካድ ነው። በእንደ

In [7]:
retriever.get_relevant_documents("ገንዘቤ ዲባባ ማን  ናት?")

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[Document(page_content='ኢትዮጵያ አይደለችም፣ችግሩ የሥርኣት ችግር ነው። የቤቴ መቃጠል ለትኋኑ በጀኝ እንዳለው\nቂል እንዳንሆን። ኢትዮጵያዊነትን መካድ የራስን ማንነት መካድ ብቻ አይደለም። አባቶችንና\nእናቶችን፣ቅድም አያቶችን ከነቅርሳቸው መካድ ነው። በእንደዚህ ኣይነቱ ክህደት ላይ የተመሰረተ\nማንነት የት ይደርሳል?ከበሽታ በቀርስ ትርፉ ምን ይሆናል? ክህደት ራሱን የቻለ በሽታ\nሲሆን ማስካድ ደግሞ የባሰ ነው። ክዶ ማስካድ ታምሜያለሁኝ፣ ታመሙ እንደማለት\nነ። አስታማሚ እንኳን እንዳይገኝ የማድረጉ ጥረት የበሽታውን ምለኣትና ፅናት \nያመለክታል።', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ግንዘቤ ዲባባ ኢትዮጵያዊት አለቃ እና መርሃግብር አስተዳዳሪ ናት። በመካከለኛ እና በርቀት የሩጫ ፍጥነት ውድድር ውስጥ \nበብዙ የዓለም አቀፍ እና የኦሊምፒክ ሜዳሊያዎች የተሸከመች እና የተለያዩ የዓለም አቀፍ እና የአፍሪካ ክርክሮች የተሸከመች\nአሸናፊ ናት። በ2015 ዓ.ም በበኢትዮጵያ በተካሄደው የአፍሪካ አለቃዎች ስፖርት አዋርድ ስነ-ስርዓት ውስጥ ከአለቃዎች አንድ እንደሆነች ተመልክታለች። \nእሷ የዲባባ ቤተሰብ ከሆነችው ትርንሳን ዲባባ እና ከእሷ እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ታሪክ የሌለው ህዝብ ታሪካዊ ስራ ለመስራት አይጓጓም። ነፃነትም የማያውቅ\nህዝብ ነፃነቱን ለማግኘት አይደክምም። ኢትዮጵያ የረጅም ዘመን ታሪክ ያላት ነፃ \nሀገር ስለሆነች ልጆቿ ታሪኳን አውቀው የነፃነቷን ጉልላት ጠብቀው

### Create langchain executor agent

In [9]:
from langchain.agents import tool

@tool
def get_link_data(link: str) -> List:
    """"""
    return None


def get_agent_executor():
    with open("../prompts/system_message.txt", "r") as file:
        system_message = file.read()

    agent_kwargs = {
    "system_message": SystemMessage(content=system_message),
    "retriever": retriever  # Pass the retriever to the agent
    }

    llm_agent = initialize_agent(
        llm=ChatOpenAI(temperature=0.1, model = 'gpt-4-1106-preview'),
        agent=AgentType.OPENAI_FUNCTIONS,
        tools=[get_link_data],
        agent_kwargs=agent_kwargs,
        verbose=True,
        max_iterations=20,
        early_stopping_method='generate'
    )

    return llm_agent


In [10]:
llm_agent = get_agent_executor()

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
llm_agent.run("ገንዘቤ ዲባባ ማን  ናት?")

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
ገንዘቤ ዲባባ እንደሚለው ስም በአሁኑ ሰአት የታወቀ ሰው ወይም እቃ አይመስለኝም። ሆኖም እሱ ሊሆን የሚችል ነገር እንደ አንድ ስም ወይም ተዋንያን ነገር ሊሆን ይችላል። እባኮትን የምትጠይቁትን በተጨማሪ ዝርዝር እንዲሁም በተመለከተ መረጃ እንዲሁም በእርስዎ የምትፈልጉት ዓይነት መረጃ ከሆነ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረ

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/chat_models/openai.py:458: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


'ገንዘቤ ዲባባ እንደሚለው ስም በአሁኑ ሰአት የታወቀ ሰው ወይም እቃ አይመስለኝም። ሆኖም እሱ ሊሆን የሚችል ነገር እንደ አንድ ስም ወይም ተዋንያን ነገር ሊሆን ይችላል። እባኮትን የምትጠይቁትን በተጨማሪ ዝርዝር እንዲሁም በተመለከተ መረጃ እንዲሁም በእርስዎ የምትፈልጉት ዓይነት መረጃ ከሆነ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ መረጃ እንዲሁም በእርስዎ የምትፈልጉት አገልግሎት ከሆነ በተጨማሪ 

{"level":"info","msg":"Created shard langchain_e8de4dcac47249479063752b57b77c8a_hSFnzwKm5OzZ in 733.647µs","time":"2024-02-01T23:17:26+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-01T23:17:26+03:00","took":105214}
{"level":"info","msg":"Created shard langchain_a14bda61e6e546c4b5467a573018b800_RQzLugQEE3Ix in 1.006484ms","time":"2024-02-01T23:17:59+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-01T23:17:59+03:00","took":116681}
{"level":"info","msg":"Created shard langchain_30a7ed705d2241cdbd1ad4f88e8983a8_SoweZG10lzhr in 881.071µs","time":"2024-02-01T23:18:13+03:00"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-02-01T23:18:13+03:00","took":83558}
{"level":"

In [13]:
llm_agent.run("who is genzeba dibaba?")



> Entering new AgentExecutor chain...
Genzebe Dibaba is an Ethiopian middle- and long-distance runner. She was born on February 8, 1991, in Bekoji, Ethiopia. Genzebe is part of an illustrious family of distance runners; she is the younger sister of three-time Olympic gold medalist Tirunesh Dibaba and Olympic silver medalist Ejegayehu Dibaba, and she is the cousin of former Olympic champion Derartu Tulu.

Genzebe has achieved significant success on the international stage, setting multiple world records across various indoor distances, including the 1500m, 3000m, and 5000m. She has also won gold medals at the World Indoor Championships and the World Championships in Athletics.

Her accomplishments have made her one of the most prominent figures in the world of athletics, and she continues to be a source of inspiration for many aspiring athletes in Ethiopia and around the world.

> Finished chain.


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/chat_models/openai.py:458: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


'Genzebe Dibaba is an Ethiopian middle- and long-distance runner. She was born on February 8, 1991, in Bekoji, Ethiopia. Genzebe is part of an illustrious family of distance runners; she is the younger sister of three-time Olympic gold medalist Tirunesh Dibaba and Olympic silver medalist Ejegayehu Dibaba, and she is the cousin of former Olympic champion Derartu Tulu.\n\nGenzebe has achieved significant success on the international stage, setting multiple world records across various indoor distances, including the 1500m, 3000m, and 5000m. She has also won gold medals at the World Indoor Championships and the World Championships in Athletics.\n\nHer accomplishments have made her one of the most prominent figures in the world of athletics, and she continues to be a source of inspiration for many aspiring athletes in Ethiopia and around the world.'

In [14]:
llm_agent.run("who is genzeba dibaba? give me your response in Amharic language")



> Entering new AgentExecutor chain...
ግንዘቤ ዲባባ ኢትዮጵያዊት አለቃ እና መርሃግብር አስተዳዳሪ ናት። በመካከለኛ እና በርቀት የሩጫ ፍጥነት ውድድር ውስጥ በብዙ የዓለም አቀፍ እና የኦሊምፒክ ሜዳሊያዎች የተሸከመች እና የተለያዩ የዓለም አቀፍ እና የአፍሪካ ክርክሮች የተሸከመች አሸናፊ ናት። በ2015 ዓ.ም በበኢትዮጵያ በተካሄደው የአፍሪካ አለቃዎች ስፖርት አዋርድ ስነ-ስርዓት ውስጥ ከአለቃዎች አንድ እንደሆነች ተመልክታለች። እሷ የዲባባ ቤተሰብ ከሆነችው ትርንሳን ዲባባ እና ከእሷ እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት 

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/chat_models/openai.py:458: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


'ግንዘቤ ዲባባ ኢትዮጵያዊት አለቃ እና መርሃግብር አስተዳዳሪ ናት። በመካከለኛ እና በርቀት የሩጫ ፍጥነት ውድድር ውስጥ በብዙ የዓለም አቀፍ እና የኦሊምፒክ ሜዳሊያዎች የተሸከመች እና የተለያዩ የዓለም አቀፍ እና የአፍሪካ ክርክሮች የተሸከመች አሸናፊ ናት። በ2015 ዓ.ም በበኢትዮጵያ በተካሄደው የአፍሪካ አለቃዎች ስፖርት አዋርድ ስነ-ስርዓት ውስጥ ከአለቃዎች አንድ እንደሆነች ተመልክታለች። እሷ የዲባባ ቤተሰብ ከሆነችው ትርንሳን ዲባባ እና ከእሷ እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወለዱት እናት እና አባት ከተወ

In [11]:
retriever.get_relevant_documents("ገንዘቤ ዲባባ ማን  ናት?")

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[Document(page_content='ዛሬ በሂወት ካላችሁ ነጋችሁን ማስተካከል ትችላላችሁ። በትናንት ውስጥ ስላጣችሁት \nነገር አታስቡ። ዛሬን ብቻ በሂወት በመኖራችሁ ፈጣሪን አመስግኑ። ዛሬ ውስጥ ጠንካራ \nከሆናችሁ ነጋችሁ የተዋበ ይሆናል። ነጋችሁ የሚጥም ይሆናል። አሁንም አሁንም አሁንም\nበሂወት መኖራችሁን ብቻ እያያችሁ አሜን በሉ። ፈጣሪንም አመስግኑ! ከተሰጣችሁ ስጦታዎች\nሁሉ ህይወታችሁ ይበልጣልና፤በሂወት ካላችሁ ደሞ በነጋችሁ ላይ ለውጥ መፍጠር ትችላላችሁ።', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ኢትዮጵያ አይደለችም፣ችግሩ የሥርኣት ችግር ነው። የቤቴ መቃጠል ለትኋኑ በጀኝ እንዳለው\nቂል እንዳንሆን። ኢትዮጵያዊነትን መካድ የራስን ማንነት መካድ ብቻ አይደለም። አባቶችንና\nእናቶችን፣ቅድም አያቶችን ከነቅርሳቸው መካድ ነው። በእንደዚህ ኣይነቱ ክህደት ላይ የተመሰረተ\nማንነት የት ይደርሳል?ከበሽታ በቀርስ ትርፉ ምን ይሆናል? ክህደት ራሱን የቻለ በሽታ\nሲሆን ማስካድ ደግሞ የባሰ ነው። ክዶ ማስካድ ታምሜያለሁኝ፣ ታመሙ እንደማለት\nነ። አስታማሚ እንኳን እንዳይገኝ የማድረጉ ጥረት የበሽታውን ምለኣትና ፅናት \nያመለክታል።', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ታሪክ የሌለው ህዝብ ታሪካዊ ስራ ለመስራት አይጓጓም። ነፃነትም የማያውቅ\nህዝብ ነፃነቱን ለማግኘት አይደክምም። ኢትዮጵያ የረጅም ዘመን ታሪክ ያላት ነፃ \nሀገር ስለሆነች ልጆቿ ታሪኳን አውቀው የነፃነቷን ጉልላት ጠብቀው/ ከተፈፀሙ ስህተቶች \nተምረው እንዲኖሩ ግዴታቸው ነው። አንድ ወጣት የሀገሩን ታሪክ ሳያውቅና ሳያጠናቅቅ\nየአውሮጳንና የአ

In [12]:
retriever.get_relevant_documents("who is genzeba dibaba?")

/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


[Document(page_content='ኢትዮጵያዊነት ኢትዮጵያዊነት ብዙ የተለያዩ ህብረተሰቦች የተዋህዱበት አካል ክፍል\nመሆን ነው። ኢትዮጵያዊነት ከጎሰኝኛት በላይና ውጭ የሆነ አጠቃላይ ማንነት ነው።\nኢትዮጵያዊነት የዚች ውጥንቅጥ መሬት ባለቤትነት ነው፣ ኢትዮጵያዊነት የደጋው\nብርድና የቆላው ሙቀት የሚገናኙበት፣ የደጋው ዝናብም ከቆላው ወንዝ ጋር\n\nየተዛመደበት ሃይል ነው። ኢትዮጵያዊነት ለሙሴ፣ ለክርስቶስና ለመሀመድ የሚጨሰው \nእጣን በእርገት መጥቶ የሚደባለቅበት እምነት ነው። ኢትዮጵያዊነት በቄጤማ ጉዝጓዝ\nላይ የሚታየው መተሳሰብና መፈቃቀር ነው። ኢትዮጵያዊነት የአስተዋይነት፣የሚዛናዊነት \nየጨዋነት ባሕርይ ነው። ኢትዮጵያዊነት ረዢምና ተፅፎ ያላለቀ ታሪክ ነው፤ ብዙ ሰዎችም\nየተሰዉለት ስሜት ነው።', metadata={'source': '../prompts/context.txt'}),
 Document(page_content='ብሩክ የሺጥላ\nበለንደን የአለም አትሌቲክስ ሻምፒዮና ኢትዮጵያዊያን አትሌቶች የሚሳተፉበት ፕሮግራም\nሀምሌ 22 ቀን 1928 ስምንት ወታደሮች በስተጀርባቸው ሃያ እርምጃ ርቀው ተንበርክከው\nበተንጠቀቅ ቆሙ። ወዲያው አዛዡ «ተኩስ» በማለት ትእዛዝ ሲሰጥ ስምንቱም ተኩሰው \nመቷቸው። ግን በስምንት ጥይት ሳይሞቱ ቀሩ። መሞታቸውንና አለመሞታቸውን ለማረጋገጥ ዶክተር ተጠራ።\nዶክተሩም እንዳልሞቱ አረጋገጠ። ከዚያም ሌላ ወታደር በስስት የሽጉጥ ጥይት ራስቅላቸውን መትቶ ገደላቸው። \nአቡነ ጴጥሮስ ለእናት ሀገራቸው ኢትዮጵያ ክብር ሲሉ የሞትን ፅዋ ተጎነጩ ። ለዛሬ እኛነታችን ያለፉት ትውልዶች \nብዙ ዋጋ ከፍለዋል እና ሊዘከሩ፣ሊመሰገኑና ሊከበሩ ይገባል። የሃላው ከሌለ የለም የፊቱ !', metadata={'source': '../prompts/contex